In [1]:
import os
from dotenv import load_dotenv
import pandas as pd
import pandas_ta as ta
from sklearn.ensemble import GradientBoostingClassifier
from emp_orderly import Strategy, EmpOrderly
from emp_orderly_types import PerpetualAssetType, Interval
import joblib
from google.cloud import pubsub_v1
import json
import time
import warnings
import nest_asyncio
import asyncio

warnings.filterwarnings('ignore')
load_dotenv()

True

In [2]:
credentials_path = 'gentle-studio-430913-r4-e2c69efea4c8.json'
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = credentials_path
project_id = 'gentle-studio-430913-r4'
publisher = pubsub_v1.PublisherClient()
pubsub_topic = f'projects/{project_id}/topics/trading_notifications'

In [3]:
def publish_trade_notification(action, symbol, price, size):
    message = {
        'action': action,
        'symbol': symbol,
        'price': price,
        'size': size,
        'timestamp': time.time()
    }
    message_json = json.dumps(message)
    message_bytes = message_json.encode('utf-8')

    try:
        publisher.publish(pubsub_topic, data=message_bytes)
        print(f"Notification sent: {action} {symbol} at {price}, size {size}")
    except Exception as err:
        print(f"Error publishing notification: {err}")

In [4]:
class GradientBoostingStrategy(Strategy):
    def init(self):
        """
        Initialize the strategy by pre-calculating technical indicators and loading a pre-trained model.
        """
        # Load pre-trained Gradient Boosting model and scaler from local files
        self.model = joblib.load('gb_model.pkl')
        self.scaler = joblib.load('scaler.pkl')

        # Prepare DataFrame with technical indicators for the strategy
        df = pd.DataFrame({
            'close': self.data.close,
            'high': self.data.high,
            'low': self.data.low
        })

        # Add technical indicators
        df['SMA_10'] = ta.sma(df['close'], length=10)
        df['SMA_20'] = ta.sma(df['close'], length=20)
        df['EMA_10'] = ta.ema(df['close'], length=10)  # Adding EMA
        macd = ta.macd(df['close'], fast=12, slow=26, signal=9)  # Adding MACD
        df['MACD'] = macd['MACD_12_26_9']
        df['MACD_signal'] = macd['MACDs_12_26_9']
        df['RSI'] = ta.rsi(df['close'], length=14)
        df['ATR'] = ta.atr(df['high'], df['low'], df['close'], length=14)
        df['volatility'] = df['close'].rolling(window=10).std()  # Adding volatility

        # Drop any rows with NaN values
        df.dropna(inplace=True)

        # Store the indicator data for future use
        self.indicator_data = df.reset_index(drop=True)

    def next(self):
        """
        Generate buy/sell signals based on the Gradient Boosting model's predictions using the latest data.
        """
        # Get the index of the last data point
        idx = len(self.data.close) - 1

        # Ensure we have enough data to make predictions
        if idx < len(self.indicator_data):
            # Extract the relevant indicators
            row = self.indicator_data.iloc[idx]
            features = [row['SMA_10'], row['SMA_20'], row['EMA_10'], row['MACD'], row['MACD_signal'], row['RSI'], row['ATR'], row['volatility']]

            # Scale the features and generate a prediction
            scaled_features = self.scaler.transform([features])
            prediction = self.model.predict(scaled_features)[0]

            # Implement buy/sell logic based on the prediction
            if prediction == 1:
                # Buy signal: close any short positions and open a long position
                if self.position.is_short:
                    self.position.close()
                if not self.position.is_long:
                    self.buy(size=1)
                    publish_trade_notification('buy', 'BTC/USDT', self.data.close[-1], 1)
            else:
                # Sell signal: close any long positions and open a short position
                if self.position.is_long:
                    self.position.close()
                if not self.position.is_short:
                    self.sell(size=1)
                    publish_trade_notification('sell', 'BTC/USDT', self.data.close[-1], 1)

In [9]:
async def run_backtest():
    # Initialize Empyreal SDK
    sdk = EmpOrderly(
        cash=70000,  # Starting capital
        commission=0.0001,  # Commission per trade
        exclusive_orders=True  # Ensure only one order at a time
    )

    # Set strategy and load data
    sdk.set_strategy(GradientBoostingStrategy)
    await sdk.load_data(
        lookback=14,
        interval=Interval.hour,
        asset=PerpetualAssetType.BTC
    )

    # Run the backtest
    stats = sdk.backtest()
    print(stats)

nest_asyncio.apply()
asyncio.get_event_loop().run_until_complete(run_backtest())

Notification sent: buy BTC/USDT at 64523.6, size 1
Notification sent: sell BTC/USDT at 63686.3, size 1
Notification sent: buy BTC/USDT at 63416.1, size 1
Notification sent: sell BTC/USDT at 63704.9, size 1
Notification sent: buy BTC/USDT at 63743.5, size 1
Notification sent: sell BTC/USDT at 64025.3, size 1
Notification sent: buy BTC/USDT at 63972.9, size 1
Notification sent: sell BTC/USDT at 63947.8, size 1
Notification sent: buy BTC/USDT at 63854.3, size 1
Notification sent: sell BTC/USDT at 63667.5, size 1
Notification sent: buy BTC/USDT at 61712.4, size 1
Notification sent: sell BTC/USDT at 60796.4, size 1
Notification sent: buy BTC/USDT at 60758.6, size 1
Notification sent: sell BTC/USDT at 62144.8, size 1
Notification sent: buy BTC/USDT at 61746.2, size 1
Notification sent: sell BTC/USDT at 61201.0, size 1
Notification sent: buy BTC/USDT at 61333.0, size 1
Notification sent: sell BTC/USDT at 61908.3, size 1
Notification sent: buy BTC/USDT at 61722.2, size 1
Notification sent: sel